In [1]:
import numpy as np
import os
import imutils
from imutils.contours import sort_contours
import numpy as np
import argparse
import cv2
import tensorflow as tf
from tensorflow import keras

# loading pre trained model
model = tf.keras.models.load_model('model-a.keras')
classnames = ['a', 'ba', 'dara', 'ei', 'ga', 'ha', 'ka', 'la', 'ma', 'na', 'nga', 'ou', 'pa', 'sa', 'ta', 'wa', 'ya']

In [67]:
import math
def image_refiner(img):
    org_size = 45
    img_size = 50
    rows,cols = img.shape
    
    if rows > cols:
        factor = org_size/rows
        rows = org_size
        cols = int(round(cols*factor))
    else:
        factor = org_size/cols
        cols = org_size
        rows = int(round(rows*factor))
    img = cv2.resize(img, (cols, rows))
    
    #get padding 
    colsPadding = (int(math.ceil((img_size-cols)/2.0)),int(math.floor((img_size-cols)/2.0)))
    rowsPadding = (int(math.ceil((img_size-rows)/2.0)),int(math.floor((img_size-rows)/2.0)))
    
    #apply apdding 
    constant_values = (255,)
    img = np.lib.pad(img,(rowsPadding,colsPadding),'constant', constant_values=constant_values)
    return img

def process():
    img = cv2.imread("As_1000.png")
    img_orig = img

    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Converts to grayscale
    ret, thresh = cv2.threshold(img, 127, 225, cv2.THRESH_BINARY) # Converts to binary
    
    # Seperate characters
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_TC89_L1)

    counter = 0
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        if (w >= 10) and (h >= 10):
            # crop image
            cropped = img[y:y+h, x:x+w]
            cropped = image_refiner(cropped)
            # apply threshold
            th, fnl = cv2.threshold(cropped, 127, 255, cv2.THRESH_BINARY)
            # apply morphology
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))
            morph = cv2.morphologyEx(fnl, cv2.MORPH_CLOSE, kernel)

            counter += 1
            # cv2.imshow("Binary Image", morph)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()

            baybayin = predict(morph)
            cv2.imwrite(f'out/{baybayin} [{counter}].png', morph)

def predict(_img):
    img = _img.reshape(-1,50,50,1)
    return classnames[np.argmax(model.predict(img))]

process()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━

In [68]:
import os
dataset = 'out'
right = 0
total = 0
for filename in os.listdir(dataset):
    total = total + 1
    filename = filename.split(' ')[0]
    if filename == 'a':
        right += 1
print(f"Right: {right} \nWrong: {total - right}\nAccuracy: {(right/total)*100}%\nDetected: {(total/1000)*100}%")

Right: 480 
Wrong: 283
Accuracy: 62.90956749672346%
Detected: 76.3%


1
3


In [ ]:
# def predict_ref():
#     file_path = "test.png"
#     test_image = tf.keras.preprocessing.image.load_img(file_path)
#     src = cv2.imread(file_path)
#     src_orig = src
#     gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    
#     cnts = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#     cnts = imutils.grab_contours(cnts)
#     cnts = sort_contours(cnts, method="left-to-right")[0]
   
#     chars = []
#     for c in cnts:
#         # compute the bounding box of the contour
#         (x, y, w, h) = cv2.boundingRect(c)
   
#         # filter out bounding boxes, ensuring they are neither too small
#         # nor too large
#         if (w >= 5 and w <= 150) and (h >= 15 and h <= 180):
#             # extract the character and threshold it to make the character
#             # appear as *white* (foreground) on a *black* background, then
#             # grab the width and height of the thresholded image
#             roi = gray[y:y + h, x:x + w]
#             thresh = cv2.threshold(roi, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
#             thresh = cv2.bitwise_not(thresh)
#             (tH, tW) = thresh.shape
#             # if the width is greater than the height, resize along the
#             # width dimension
#             if tW > tH:
#                 thresh = imutils.resize(thresh, width=50)
#             # otherwise, resize along the height
#             else:
#                 thresh = imutils.resize(thresh, height=50)
#             # re-grab the image dimensions (now that its been resized)
#             # and then determine how much we need to pad the width and
#             # height such that our image will be 50x50
#             (tH, tW) = thresh.shape
#             dX = int(max(0, 50 - tW) / 2.0)
#             dY = int(max(0, 50 - tH) / 2.0)
#             # pad the image and force 32x32 dimensions
#             padded = cv2.copyMakeBorder(thresh, top=dY, bottom=dY,
#                                         left=dX, right=dX, borderType=cv2.BORDER_CONSTANT,
#                                         value=(255, 255, 255))
#             padded = cv2.resize(roi, (50, 50))
#             # prepare the padded image for classification via our
#             # handwriting OCR model
#             padded = padded.astype("float32") / 255.0
#             padded = np.expand_dims(padded, axis=-1)
#             # update our list of characters that will be OCR'd
#             chars.append((padded, (x, y, w, h)))
#             # -------------------------------------------------------------
#             cv2.rectangle(src_orig,(x,y),(x+w,y+h),(0,255,0),2)
#             cv2.imwrite('output.jpg', roi)
#             # -------------------------------------------------------------
#     boxes = [b[1] for b in chars]
   
#     chars = np.array([c[0] for c in chars], dtype="float32")
#     # OCR the characters using our handwriting recognition model
#     preds = model.predict(chars)
#     # define the list of label names
#     labelNames = ['a', 'ba', 'dara', 'ei', 'ga', 'ha', 'ka', 'la', 'ma', 'na', 'nga', 'ou', 'pa', 'sa', 'ta', 'wa', 'ya']
#     output = ""
#     for (pred, (x, y, w, h)) in zip(preds, boxes):
#         i = np.argmax(pred)
#         prob = pred[i]
#         label = labelNames[i]
#         output += (label + " ")

#     print("output" , label)
# predict_ref()

In [3]:
import cv2
import math
import numpy as np
def image_refiner(img):
    org_size = 40
    img_size = 50
    rows,cols = img.shape
    
    if rows > cols:
        factor = org_size/rows
        rows = org_size
        cols = int(round(cols*factor))
    else:
        factor = org_size/cols
        cols = org_size
        rows = int(round(rows*factor))
    img = cv2.resize(img, (cols, rows))
    
    #get padding 
    colsPadding = (int(math.ceil((img_size-cols)/2.0)),int(math.floor((img_size-cols)/2.0)))
    rowsPadding = (int(math.ceil((img_size-rows)/2.0)),int(math.floor((img_size-rows)/2.0)))
    
    #apply apdding 
    constant_values = (0,)
    img = np.lib.pad(img,(rowsPadding,colsPadding),'constant', constant_values=constant_values)
    return img

img = cv2.imread("Dot_500.png")
img_orig = img
# Seperate characters
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Converts to grayscale

counter = 0
for x in range(0, 1150, 50):
    for y in range(0, 1150, 50):
        cropped = img[y:y+50, x:x+50]
       
        cv2.imwrite(f'dots/Dot [{counter}].png', cropped)
        counter += 1